In [1]:
import pandas as pd
df = pd.read_excel(r'C:\Users\LENOVO\Desktop\Python\dataset_for_deepLearning\sentiment_analysis_for_financial_news\all-data.xlsx', engine='openpyxl')
df.dropna(inplace = True)
df.shape

(4846, 2)

In [31]:
df.head()

,label,text
0,1,"According to Gran , the company has no plans t..."
1,1,Technopolis plans to develop in stages an area...
2,0,The international electronic industry company ...
3,2,With the new production plant the company woul...
4,2,According to the company 's updated strategy f...


In [2]:
etiket=df.iloc[:,0:1].values
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
etiket[:,0]=le.fit_transform(df.iloc[:,0])
ohe=preprocessing.OneHotEncoder()
etiket=ohe.fit_transform(etiket).toarray()

In [3]:
X=df.iloc[:,1:2].values
print(type(X))
print(type(etiket))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, etiket, test_size=0.33, random_state=42)

In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3246, 1)
(3246, 3)
(1600, 1)
(1600, 3)


In [6]:
texts=[]
for x in range(0,X_train.shape[0]):
    texts.append(X_train[x,0])

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
maxlen=100
training_samples=2500
validation_samples=746
max_words=10000
tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)
word_index=tokenizer.word_index
print("Found %s unique tokens."%len(word_index))
data=pad_sequences(sequences,maxlen=maxlen)
labels=np.asarray(y_train)
print("Shape of data tensor:",data.shape)
print("Shape of label tensor:",labels.shape)
indices=np.arange(data.shape[0])
np.random.shuffle(indices)
data=data[indices]
labels=labels[indices]
x_train=data[:training_samples]
y_train=labels[:training_samples]
x_val=data[training_samples:training_samples+validation_samples]
y_val=labels[training_samples:training_samples+validation_samples]

Using TensorFlow backend.


Found 8185 unique tokens.
Shape of data tensor: (3246, 100)
Shape of label tensor: (3246, 3)


In [8]:
import os
glove_dir="C:/Users/LENOVO/Desktop/Python/dataset_for_deepLearning/Imdb/glove.6B"
embeddings_index={}
f=open(os.path.join(glove_dir,"glove.6B.100d.txt"),encoding="utf8")
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype="float32")
    embeddings_index[word]=coefs
f.close()
print("found %s word vectors"%len(embeddings_index))

found 400000 word vectors


In [9]:
embedding_dim=100
embedding_matrix=np.zeros((max_words,embedding_dim))
for word, i in word_index.items():
    if i<max_words:
        embedding_vector=embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector

In [24]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, SimpleRNN, LSTM
model=Sequential()
model.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model.add(LSTM(32))
model.add(Dense(3,activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 1,017,123
Trainable params: 1,017,123
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable=False

In [27]:
model.compile(optimizer="rmsprop",
             loss="categorical_crossentropy",
             metrics=["acc"])
history=model.fit(x_train,y_train,
                 epochs=100,
                 batch_size=32,
                 validation_data=(x_val,y_val))

Train on 2500 samples, validate on 746 samples
Epoch 1/100
2500/2500 [==============================] - 7s - loss: 0.4697 - acc: 0.7992 - val_loss: 0.7936 - val_acc: 0.6836
Epoch 2/100
2500/2500 [==============================] - 7s - loss: 0.4480 - acc: 0.8052 - val_loss: 0.7313 - val_acc: 0.7252
Epoch 3/100
2500/2500 [==============================] - 7s - loss: 0.4189 - acc: 0.8220 - val_loss: 0.6902 - val_acc: 0.7306
Epoch 4/100
2500/2500 [==============================] - 6s - loss: 0.4059 - acc: 0.8252 - val_loss: 0.8213 - val_acc: 0.6542
Epoch 5/100
2500/2500 [==============================] - 6s - loss: 0.3813 - acc: 0.8420 - val_loss: 0.6942 - val_acc: 0.7225
Epoch 6/100
2500/2500 [==============================] - 6s - loss: 0.3602 - acc: 0.8576 - val_loss: 0.7799 - val_acc: 0.7265
Epoch 7/100
2500/2500 [==============================] - 6s - loss: 0.3436 - acc: 0.8540 - val_loss: 0.8434 - val_acc: 0.6568
Epoch 8/100
2500/2500 [==============================] - 6s - loss: 0.3

2500/2500 [==============================] - 5s - loss: 0.0155 - acc: 0.9948 - val_loss: 1.7858 - val_acc: 0.7359
Epoch 66/100
2500/2500 [==============================] - 6s - loss: 0.0209 - acc: 0.9944 - val_loss: 1.8890 - val_acc: 0.7198
Epoch 67/100
2500/2500 [==============================] - 6s - loss: 0.0108 - acc: 0.9972 - val_loss: 2.0147 - val_acc: 0.6984
Epoch 68/100
2500/2500 [==============================] - 5s - loss: 0.0136 - acc: 0.9968 - val_loss: 1.8412 - val_acc: 0.7292
Epoch 69/100
2500/2500 [==============================] - 5s - loss: 0.0094 - acc: 0.9976 - val_loss: 1.8743 - val_acc: 0.7145
Epoch 70/100
2500/2500 [==============================] - 6s - loss: 0.0133 - acc: 0.9976 - val_loss: 2.0283 - val_acc: 0.7158
Epoch 71/100
2500/2500 [==============================] - 6s - loss: 0.0115 - acc: 0.9984 - val_loss: 1.9702 - val_acc: 0.7131
Epoch 72/100
2500/2500 [==============================] - 7s - loss: 0.0226 - acc: 0.9944 - val_loss: 1.8752 - val_acc: 0.70

In [28]:
test_texts=[]
for x in range(0,X_test.shape[0]):
    test_texts.append(X_test[x,0])

In [29]:
sequences=tokenizer.texts_to_sequences(test_texts)
x_test=pad_sequences(sequences,maxlen=maxlen)
y_test=np.asarray(y_test)

In [30]:
model.evaluate(x_test,y_test)

1600/1600 [==============================] - 0s     


[1.8539171874523164, 0.75875]